# Audio MNIST

In [1]:
from setup import Setup
from utilities import Utilities
import glob
import os
import pandas as pd
from data_processing import DataProcessing
from data_visualization import DataVisualization

# Initialize the setup class with the config file path
SU = Setup(cfg_filepath = 'config.yaml')

# Get the paths
source_path = SU.source_path
meta_data_path = SU.source_meta_path
destination_path = SU.destination_path
plot_path = SU.plot_path
meta_data = SU.meta_data

# Initialize the Utilities class
UT = Utilities()

# Define the target sample rate
target_sr = 8000

# Initialize the DataProcessing class with the destination path, and target sample rate
DP = DataProcessing(destination_path, target_sr)

# Initialize the DataVisualization class with the target sample rate
DV = DataVisualization()

# Create empty dataframe with column names
columns=['mean', 'std', 'avg', 'median', 'min', 'max', 'skewness', 'kurtosis', 'gender', 'digit']
df = UT.create_dataframe(columns)

# Loop over the audio recordings in the source path
all_folders = len(next(os.walk(source_path))[1])+1
for i in range(1, 2): # instert all_folders when done testing
    src_temp = os.path.join(source_path, f"{i:02d}")
    filepath_filename = sorted(glob.glob(os.path.join(src_temp, "*.wav")))
    for file in filepath_filename:
        # Split file string
        dig, vp, rep = file.rstrip(".wav").split("/")[-1].split("_")

        # read audio data
        fs, data = UT.read_audio(file)

        # plot audio signal
        plot_name = f"{dig[-1]}_{vp}_{rep}.png"
        #DV.plot_signal(fs, data, plot_path, plot_name)

        # play audio signal
        #DV.play_audio(file)

        # resample audio data
        data = DP.resample_data(fs, data, target_sr)

        # zero padding audio data
        data = DP.zero_pad(data, target_sr)

        # FFT audio data
        data = DP.fft_data(data)

        # Feature creation
        features = DP.feature_creation(data)

        # Normalize features
        n_features = DP.normalize_features(features)

        # Add gender and digit label
        features = DP.add_gender(n_features, meta_data[vp]["gender"])
        features = DP.add_digit(n_features, dig[-1])

        # Append new dict values to the DataFrame
        df = df.append(features, ignore_index=True)

# Save data to CSV
csv_name = "audio_data.csv"
UT.save_df_to_csv(df, destination_path, csv_name)